# Title generation and summarization

In [1]:
# !pip install -U -r ../requirements.txt

## Import libraries

In [1]:
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import os
import pandas as pd

from langchain import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,)

# from dotenv import load_dotenv

# load_dotenv()

# openai.api_key = os.environ["OPENAI_API_KEY"]
# openai.api_base = os.environ["OPENAI_API_BASE"]
# openai.api_type = "azure"
# openai.api_version = "2023-05-15"
# chat_deployment = os.environ["OPENAI_CHAT_DEPLOYMENT"]
# OPENAI_EMBEDDING_DEPLOYMENT = os.environ["OPENAI_EMBEDDING_DEPLOYMENT"]

## Define Azure Openai

In [4]:
llm =  AzureChatOpenAI(
            temperature=0.0,
            deployment_name='gpt-4',
            openai_api_key=openai.api_key,
            openai_api_base=openai.api_base,
            openai_api_type="azure",
            openai_api_version="2023-05-15",
        )

embeddings = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_DEPLOYMENT,
    deployment=OPENAI_EMBEDDING_DEPLOYMENT,
    openai_api_key= openai.api_key,
    openai_api_base=openai.api_base,
    openai_api_type=openai.api_type,
    openai_api_version=openai.api_version,
    chunk_size=16,
)

## Install document loader dependencies

In [2]:
# !pip install "unstructured[all-docs]"

## Load the data and chain

In [2]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain

In [3]:
loader = UnstructuredFileLoader(
    "../data/tslaletter.htm - Generated by SEC Publisher for SEC Filing.pdf"
)
docs = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jarvixwang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jarvixwang/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
# print(len(docs))
# for d in docs:
    # print(d)
print(docs[0].page_content)


10/18/23, 11:42 AM

tslaletter.htm - Generated by SEC Publisher for SEC Filing

PX14A6N 1 tslaletter.htm

SECURITIES & EXCHANGE COMMISSION WASHINGTON, D.C. 20549

NOTICE OF EXEMPT SOLICITATION (VOLUNTARY SUBMISSION)

NAME OF REGISTRANT: Tesla, Inc. NAME OF PERSON RELYING ON EXEMPTION: SOC Investment Group ADDRESS OF PERSON RELYING ON EXEMPTION: 1900 L Street, NW, Suite 900, Washington, DC 20036 Written materials are submitted pursuant to Rule 14a-6(1) promulgated under the Securities Exchange Act of 1934: ______________________________________________________________________________________________________________________________________

April 24, 2023

Dear Fellow Tesla Shareholders:

We are writing to urge you to vote AGAINST the election of J.B. Straubel to the Tesla Board of Directors (the “Board”) on May 16th. Mr. Straubel has been nominated to replace independent director Hiro Mizuno. While we appreciate the contributions Mr. Straubel made as co-founder and long-time Chief Techn

### Define Stuff/Map-Reduce/Refine Chain

In [11]:
from IPython.display import display, Markdown, HTML, JSON

In [7]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Define prompt
prompt_template = """Write a concise summary of the following text, answer in Chinese:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)
stuff_chain_summary = (stuff_chain.run(docs))
display(Markdown(stuff_chain_summary))

特斯拉股东呼吁投票反对J.B. Straubel加入董事会，认为他是公司内部人士，不适合加入独立性不足的董事会。目前，董事会与CEO埃隆·马斯克关系密切，包括他的兄弟金博尔·马斯克以及与他有私人友谊的伊拉·埃伦普雷斯和詹姆斯·默多克。此外，特斯拉董事会的薪酬水平过高，可能影响其客观性和对CEO的挑战。用另一位内部人士替换独立董事可能会加剧现有问题。

In [8]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain import hub

# Map阶段: 将doc进行summary
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
# map_prompt = PromptTemplate.from_template(map_template)
map_prompt = hub.pull("rlm/map-prompt")
map_chain = LLMChain(llm=llm, prompt=map_prompt)


# Reduce: 拿到所有summary再做最后一次summary
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes. Answer in Chinese. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
# reduce_prompt = hub.pull("rlm/reduce-prompt")
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)


# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=100
)
split_docs = text_splitter.split_documents(docs)

map_reduce_summary = map_reduce_chain.run(split_docs)

display(Markdown(map_reduce_summary))

1. 特斯拉董事会独立性不足，监管不力，对CEO马斯克过于迁就。
2. 马斯克的多重商业冒险和对特斯拉的兼职承诺。
3. 马斯克巨额薪酬待遇及其对股东的影响。
4. 需要强大、独立的董事会来平衡马斯克的行为。
5. 马斯克极端言论对特斯拉品牌声誉的负面影响。
6. J.B. Straubel被提名为特斯拉董事会成员，但由于与马斯克的密切关系引发担忧。
7. 特斯拉弗里蒙特工厂存在种族歧视、骚扰和性骚扰指控，以及马斯克对这些指控的回应。
8. 马斯克在推特上的争议性言论及其对特斯拉品牌形象的影响。

In [10]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Chinese"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

print(result["output_text"])

SOC Investment Group敦促特斯拉股东投票反对公司联合创始人J.B. Straubel进入董事会，原因是担忧董事会的独立性。该集团认为，用内部人士取代独立董事将加剧现有问题，包括高CEO和董事薪酬、薄弱的质押政策以及宽松的顶层氛围。如果Straubel当选，该集团估计特斯拉董事会只有38%的独立性。特斯拉董事会已经过于顺从CEO马斯克，没有对他在多家公司兼职的问题进行监督。此外，特斯拉董事会允许马斯克将其特斯拉股票的25%作为个人债务抵押品，给股东带来风险。特斯拉需要一个强大、独立的董事会来平衡马斯克的行为。此外，特斯拉在过去几年中股价波动巨大，部分原因可能与马斯克的行为有关。特斯拉还面临着来自员工的种族歧视和性骚扰指控，这些问题在现有董事会的监督下持续发酵。根据纽约时报2022年底的一项调查，董事会主席Denholm自2020年以来已通过出售作为报酬的特斯拉股票赚取了2.8亿美元，而Murdoch自2017年成为特斯拉董事以来获得了1100万美元的报酬，主要是股票期权。特斯拉董事会需要有能力对抗强势创始人CEO的独立董事，而Straubel似乎并不具备这种能力。他与马斯克的个人关系可能使他在董事会中缺乏独立性。因此，投资者呼吁反对J.B. Straubel的选举。


In [11]:
refine_summary = result["output_text"]

In [29]:
chain = load_summarize_chain(llm, chain_type="refine")
refine_summary = chain.run(split_docs)
display(Markdown(refine_summary))

On September 5, 2023, Tesla CFO Vaibhav Taneja reported transactions involving the acquisition and disposal of company shares. Taneja acquired 4,000 shares at $18.22 each and disposed of the same amount at $250.018 each. Additionally, Taneja acquired 388 shares at no cost and disposed of 176.75 shares at $249.622 each. The transactions were filed with the United States Securities and Exchange Commission. These transactions were effected pursuant to a Rule 10b5-1 trading plan adopted by Taneja on February 14, 2023. Out of the 4,000 shares sold, approximately 1,913 shares were sold to cover the exercise price and tax withholding obligations related to the exercise of stock options.

In [12]:
cod_prompt_template = """
Article: {ARTICLE} You will generate increasingly concise, entity-dense summaries of the above article.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is:

relevant to the main story,
specific yet concise (5 words or fewer),
novel (not in the previous summary),
faithful (present in the article),
anywhere (can be located anywhere in the article).
Guidelines:

The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article.
Missing entities can appear anywhere in the new summary.
Never drop entities from the previous summary. If space cannot be made, add fewer new entities.
Remember, use the exact same number of words for each summary. Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Answer in Chinese.
"""

In [13]:
cod_prompt_template_chinese = """
您需要为以下文章生成越来越简洁、实体丰富的摘要：{ARTICLE}

重复以下2个步骤5次:

步骤1:从文章中找出1-3个有用且与先前生成的摘要不同的实体。用分号分隔。
步骤2:编写一个新的、实体丰富的摘要,长度与之前的摘要相同,覆盖先前摘要的所有实体和细节以及新增实体。

缺失实体应满足以下条件：

1. 与主要内容相关；
2. 简洁明了，五个字以内；
3. 新颖度高，摘要中尚未包含；
4. 忠实于文章，确实存在于原文中；
5. 可以出现在摘要中的任意位置。

请遵循以下指导原则：

1. 首个摘要较长(4-5句,约80字),但信息量较低,仅包含标记为缺失的实体。请使用较为冗长的表述和补充内容(如"本文讨论了")使其达到约80字。
2. 删减摘要中的废话，增加实体信息。
3. 用融合、压缩以及优化方式删除像“本文讨论了”这样的非信息性短语。
4. 摘要应具有高度的密度和简洁性，且内容自包含，即使未看原文也能理解。
5. 新实体可以出现在新摘要的任意位置。
6. 摘要中的实体不重复。如空间不足，则添加较少的新实体。

请以JSON格式回答。此JSON为一个列表,长度为5,每个元素是一个dict, key为"Missing_Entities"和"Denser_Summary"。
"""

In [14]:
#cod_prompt = hub.pull("lawwu/chain_of_density")
cod_prompt = PromptTemplate.from_template(cod_prompt_template_chinese)

cod_prompt_chat = ChatPromptTemplate.from_template(cod_prompt_template_chinese)

cod_chain = LLMChain(llm=llm, prompt=cod_prompt_chat)

result = cod_chain.invoke({"ARTICLE": docs[0].page_content})

display(Markdown(result['text']))

[
{
  "Missing_Entities": "SOC Investment Group; Robyn Denholm; Hiro Mizuno",
  "Denser_Summary": "SOC Investment Group敦促Tesla股东反对J.B. Straubel加入董事会。他们认为，由于Straubel与CEO Elon Musk关系密切，他不适合取代独立董事Hiro Mizuno。目前，董事会与Musk关系密切，包括他的兄弟Kimbal Musk以及Ira Ehrenpreis和James Murdoch。此外，董事会主席Robyn Denholm的薪酬也引起了关注。"
},
{
  "Missing_Entities": "Joseph Gebbia; TruthGPT; Elon Musk",
  "Denser_Summary": "Tesla董事会独立性不足，Joseph Gebbia等董事与Elon Musk关系密切。Musk在Tesla、SpaceEx、Twitter等多家公司担任CEO，并创立了TruthGPT等创业公司。董事会对Musk的监管不力，导致了诸多丑闻，损害了Tesla的品牌和运营。"
},
{
  "Missing_Entities": "YouGov; Morning Consult; Delaware Court",
  "Denser_Summary": "根据YouGov和Morning Consult的调查，Tesla品牌声誉下滑，与Musk在Twitter上的极端言论有关。此外，特拉华州法院对Musk的巨额薪酬计划进行审查，认为其可能存在利益冲突。董事会独立性不足，无法对Musk进行有效监管。"
},
{
  "Missing_Entities": "Fremont工厂; California Department; Elon Musk",
  "Denser_Summary": "Tesla Fremont工厂存在恶劣的工作环境，多名员工提起诉讼。加州公平住房与就业部门对Tesla提起诉讼，称黑人员工在工厂中遭受敌对环境。此外，Musk在SpaceEx面临性骚扰指控。董事会独立性不足，无法对Musk进行有效监管。"
},
{
  "Missing_Entities": "Time Magazine; Elon Musk; J.B. Straubel",
  "Denser_Summary": "Time Magazine采访中，J.B. Straubel表示他与Elon Musk关系密切，但不愿过多谈论。Straubel被提名加入Tesla董事会，但由于与Musk关系密切，可能无法对其进行有效监管。董事会独立性不足，需要进行全面改革。"
}
]

In [118]:
cod_prompt.template

'\nArticle: {ARTICLE} You will generate increasingly concise, entity-dense summaries of the above article.\n\nRepeat the following 2 steps 5 times.\n\nStep 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.\n\nA missing entity is:\n\nrelevant to the main story,\nspecific yet concise (5 words or fewer),\nnovel (not in the previous summary),\nfaithful (present in the article),\nanywhere (can be located anywhere in the article).\nGuidelines:\n\nThe first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.\nMake every word count: rewrite the previous summary to improve flow and make space f

In [16]:
import json
dense_json = json.loads(result["text"])

In [17]:
for item in dense_json:
    print(item['Missing_Entities'])
    print(item['Denser_Summary'])
    print("\n")

SOC Investment Group; Robyn Denholm; Hiro Mizuno
SOC Investment Group敦促Tesla股东反对J.B. Straubel加入董事会。他们认为，由于Straubel与CEO Elon Musk关系密切，他不适合取代独立董事Hiro Mizuno。目前，董事会与Musk关系密切，包括他的兄弟Kimbal Musk以及Ira Ehrenpreis和James Murdoch。此外，董事会主席Robyn Denholm的薪酬也引起了关注。


Joseph Gebbia; TruthGPT; Elon Musk
Tesla董事会独立性不足，Joseph Gebbia等董事与Elon Musk关系密切。Musk在Tesla、SpaceEx、Twitter等多家公司担任CEO，并创立了TruthGPT等创业公司。董事会对Musk的监管不力，导致了诸多丑闻，损害了Tesla的品牌和运营。


YouGov; Morning Consult; Delaware Court
根据YouGov和Morning Consult的调查，Tesla品牌声誉下滑，与Musk在Twitter上的极端言论有关。此外，特拉华州法院对Musk的巨额薪酬计划进行审查，认为其可能存在利益冲突。董事会独立性不足，无法对Musk进行有效监管。


Fremont工厂; California Department; Elon Musk
Tesla Fremont工厂存在恶劣的工作环境，多名员工提起诉讼。加州公平住房与就业部门对Tesla提起诉讼，称黑人员工在工厂中遭受敌对环境。此外，Musk在SpaceEx面临性骚扰指控。董事会独立性不足，无法对Musk进行有效监管。


Time Magazine; Elon Musk; J.B. Straubel
Time Magazine采访中，J.B. Straubel表示他与Elon Musk关系密切，但不愿过多谈论。Straubel被提名加入Tesla董事会，但由于与Musk关系密切，可能无法对其进行有效监管。董事会独立性不足，需要进行全面改革。




In [18]:
dense_summary = dense_json[-1]['Denser_Summary']

In [19]:
display(Markdown(dense_json[-1]['Denser_Summary']))

Time Magazine采访中，J.B. Straubel表示他与Elon Musk关系密切，但不愿过多谈论。Straubel被提名加入Tesla董事会，但由于与Musk关系密切，可能无法对其进行有效监管。董事会独立性不足，需要进行全面改革。

In [23]:
title_generation_template = """
You are a financial journalist writing a title for the following article:
{SUMMARY}
Guidelines:
1. The title should be 10 words or fewer.
2. The title should become highly dense and concise yet self-contained, i.e., easily understood without the article.
3. The title should be written in Chinese.
Let's think step by step:
"""

In [24]:
title_prompt = PromptTemplate.from_template(title_generation_template)
title_chain = LLMChain(llm=llm, prompt=title_prompt)

summary_list = [
    {
        "Method": "StuffDocumentsChain",
        "Summary": stuff_chain_summary,
    },
    {
        "Method": "MapReduceDocumentsChain",
        "Summary": map_reduce_summary,
    },
    {
        "Method": "RefineDocumentsChain",
        "Summary": refine_summary,
    },
    {
        "Method": "ChainOfDensity",
        "Summary": dense_summary,
    },
]
for summary in summary_list:
    result = title_chain.invoke({"SUMMARY": summary['Summary']})
    display(Markdown(result['text']))
    summary['Title'] = result['text']

特斯拉股东反对内部人士Straubel加入董事会

特斯拉董事会监管不力，马斯克行为争议影响品牌声誉

特斯拉独立性受威胁？投资集团反对创始人Straubel入董事会

特斯拉董事会独立性不足，Straubel提名引关注

In [25]:
## Create a pandas dataframe, with header 'Method','Summary'
import pandas as pd

summary_list = [
    {'Method':'StuffDocumentsChain','Summary':stuff_chain_summary},
    {'Method':'MapReduceDocumentsChain','Summary':map_reduce_summary},
    {'Method':'RefineDocumentsChain','Summary':refine_summary},
    {'Method':'Chain of Density','Summary':dense_summary}
]
df = pd.DataFrame(summary_list, columns = ['Method','Summary'])


In [136]:
df.to_excel('../data/summary.xlsx', index=False)

In [27]:
pd.set_option('max_colwidth', 800)
df

,Method,Summary
0,StuffDocumentsChain,特斯拉股东呼吁投票反对J.B. Straubel加入董事会，认为他是公司内部人士，不适合加入独立性不足的董事会。目前，董事会与CEO埃隆·马斯克关系密切，包括他的兄弟金博尔·马斯克以及与他有私人友谊的伊拉·埃伦普雷斯和詹姆斯·默多克。此外，特斯拉董事会的薪酬水平过高，可能影响其客观性和对CEO的挑战。用另一位内部人士替换独立董事可能会加剧现有问题。
1,MapReduceDocumentsChain,1. 特斯拉董事会独立性不足，监管不力，对CEO马斯克过于迁就。\n2. 马斯克的多重商业冒险和对特斯拉的兼职承诺。\n3. 马斯克巨额薪酬待遇及其对股东的影响。\n4. 需要强大、独立的董事会来平衡马斯克的行为。\n5. 马斯克极端言论对特斯拉品牌声誉的负面影响。\n6. J.B. Straubel被提名为特斯拉董事会成员，但由于与马斯克的密切关系引发担忧。\n7. 特斯拉弗里蒙特工厂存在种族歧视、骚扰和性骚扰指控，以及马斯克对这些指控的回应。\n8. 马斯克在推特上的争议性言论及其对特斯拉品牌形象的影响。
2,RefineDocumentsChain,SOC Investment Group敦促特斯拉股东投票反对公司联合创始人J.B. Straubel进入董事会，原因是担忧董事会的独立性。该集团认为，用内部人士取代独立董事将加剧现有问题，包括高CEO和董事薪酬、薄弱的质押政策以及宽松的顶层氛围。如果Straubel当选，该集团估计特斯拉董事会只有38%的独立性。特斯拉董事会已经过于顺从CEO马斯克，没有对他在多家公司兼职的问题进行监督。此外，特斯拉董事会允许马斯克将其特斯拉股票的25%作为个人债务抵押品，给股东带来风险。特斯拉需要一个强大、独立的董事会来平衡马斯克的行为。此外，特斯拉在过去几年中股价波动巨大，部分原因可能与马斯克的行为有关。特斯拉还面临着来自员工的种族歧视和性骚扰指控，这些问题在现有董事会的监督下持续发酵。根据纽约时报2022年底的一项调查，董事会主席Denholm自2020年以来已通过出售作为报酬的特斯拉股票赚取了2.8亿美元，而Murdoch自2017年成为特斯拉董事以来获得了1100万美元的报酬，主要是股票期权。特斯拉董事会需要有能力对抗强势创始人CEO的独立董事，而Straubel似乎并不具备这种能力。他与马斯克的个人关系可能使他在董事会中缺乏独立性。因此，投资者呼吁反对J.B. Straubel的选举。
3,Chain of Density,Time Magazine采访中，J.B. Straubel表示他与Elon Musk关系密切，但不愿过多谈论。Straubel被提名加入Tesla董事会，但由于与Musk关系密切，可能无法对其进行有效监管。董事会独立性不足，需要进行全面改革。


In [128]:
Markdown(docs[0].page_content)

2023/9/19 15:21

SEC FORM 4

SEC Form 4

FORM 4

Check this box if no longer subject to Section 16. Form 4 or Form 5 obligations may continue. See Instruction 1(b).

UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549

STATEMENT OF CHANGES IN BENEFICIAL OWNERSHIP

Filed pursuant to Section 16(a) of the Securities Exchange Act of 1934 or Section 30(h) of the Investment Company Act of 1940

OMB APPROVAL

OMB Number: Estimated average burden

3235-0287

hours per response:

0.5

X

Check this box to indicate that a transaction was made pursuant to a contract, instruction or written plan for the purchase or sale of equity securities of the issuer that is intended to satisfy the affirmative defense conditions of Rule 10b5- 1(c). See Instruction 10.

1. Name and Address of Reporting Person* Taneja Vaibhav

(Last)

(First)

C/O TESLA, INC. 1 TESLA ROAD

(Middle)

2. Issuer Name and Ticker or Trading Symbol Tesla, Inc. [ TSLA ]

3. Date of Earliest Transaction (Month/Day/Year) 09/05/2023

5. Relationship of Reporting Person(s) to Issuer (Check all applicable) Director Officer (give title below)

X

10% Owner Other (specify below)

Chief Financial Ofﬁcer

(Street) AUSTIN

TX

78725

4. If Amendment, Date of Original Filed (Month/Day/Year)

6. Individual or Joint/Group Filing (Check Applicable Line)

X

Form filed by One Reporting Person Form filed by More than One Reporting Person

(City)

(State)

(Zip)

Table I - Non-Derivative Securities Acquired, Disposed of, or Beneficially Owned

1. Title of Security (Instr. 3)

2. Transaction Date (Month/Day/Year)

2A. Deemed Execution Date, if any (Month/Day/Year)

3. Transaction Code (Instr. 8)

Code

V

4. Securities Acquired (A) or Disposed Of (D) (Instr. 3, 4 and 5)

Amount

(A) or (D)

Price

5. Amount of Securities Beneficially Owned Following Reported Transaction(s) (Instr. 3 and 4)

6. Ownership Form: Direct (D) or Indirect (I) (Instr. 4)

7. Nature of Indirect Beneficial Ownership (Instr. 4)

Common Stock

09/05/2023

M(1)

4,000

A

$18.22

108,504.25

D

Common Stock

09/05/2023

S(1)

4,000(2)

D

$250.018(3)

104,504.25

D

Common Stock

09/05/2023

M(4)

388

A

$0.0

104,892.25

D

Common Stock

09/06/2023

S(5)

176.75

D

$249.622

104,715.5

D

Table II - Derivative Securities Acquired, Disposed of, or Beneficially Owned (e.g., puts, calls, warrants, options, convertible securities)

1. Title of Derivative Security (Instr. 3)

2. Conversion or Exercise Price of Derivative Security

3. Transaction Date (Month/Day/Year)

3A. Deemed Execution Date, if any (Month/Day/Year)

4. Transaction Code (Instr. 8)

5. Number of Derivative Securities Acquired (A) or Disposed of (D) (Instr. 3, 4 and 5)

6. Date Exercisable and Expiration Date (Month/Day/Year)

7. Title and Amount of Securities Underlying Derivative Security (Instr. 3 and 4)

8. Price of Derivative Security (Instr. 5)

9. Number of derivative Securities Beneficially Owned Following Reported Transaction(s) (Instr. 4)

10. Ownership Form: Direct (D) or Indirect (I) (Instr. 4)

11. Nature of Indirect Beneficial Ownership (Instr. 4)

Code

V

(A)

(D)

Date Exercisable

Expiration Date

Title

Amount or Number of Shares

Non- Qualiﬁed Stock Option (right to buy)

$18.22

09/05/2023

M(1)

4,000

(6)

04/19/2029

Common Stock

4,000

$0.0

808,920

D

Restricted Stock Unit

$0.0

09/05/2023

M

388

(7)

(7)

Common Stock

388

$0.0

388

D

Explanation of Responses:

1. The transactions reported on this Form 4 were effected pursuant to a Rule 10b5-1 trading plan adopted by the reporting person on February 14, 2023.

2. Out of the total of 4,000 shares sold, approximately 1,913 shares were sold to cover the exercise price of, and to satisfy the reporting person's tax withholding obligations related to the exercise of stock options to purchase 4,000 shares as reported herein.

3. The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $250.000 to $250.060, inclusive. The reporting person undertakes to provide Tesla, Inc., any security holder of Tesla, Inc. or the staff of the Securities and Exchange Commission, upon request, full information regarding the number of shares sold at each separate price within the range set forth in this footnote. 4. Shares of the Issuer's common stock were issued to the reporting person upon the vesting of restricted stock units on September 5, 2023. 5. PURSUANT TO THE ISSUER'S EQUITY PLAN AND POLICIES, THESE SHARES OF COMMON STOCK WERE AUTOMATICALLY WITHHELD AND SOLD BY THE ISSUER TO SATISFY THE REPORTING PERSON'S TAX WITHHOLDING OBLIGATIONS RELATED TO THE VESTING OF RESTRICTED STOCK UNITS REPORTED HEREIN. 6. 1/8th of the shares subject to the option became vested and exercisable on September 13, 2019, and an additional 1/48th of the shares subject to the option vested each month thereafter, so that all such shares subject to this option became fully vested on March 13, 2023. 7. 1/20th of the total restricted stock units initially subject to this award vested on March 5, 2019, and 1/20th of the total units initially subject to this award vest every three months thereafter, so that all such shares subject to this award will be fully vested by December 5, 2023.

By: Aaron Beckman, Power of Attorney For: Vaibhav Taneja ** Signature of Reporting Person

09/07/2023

Date

Reminder: Report on a separate line for each class of securities beneficially owned directly or indirectly. * If the form is filed by more than one reporting person, see Instruction 4 (b)(v). ** Intentional misstatements or omissions of facts constitute Federal Criminal Violations See 18 U.S.C. 1001 and 15 U.S.C. 78ff(a). Note: File three copies of this Form, one of which must be manually signed. If space is insufficient, see Instruction 6 for procedure.

Persons who respond to the collection of information contained in this form are not required to respond unless the form displays a currently valid OMB Number.

https://www.sec.gov/Archives/edgar/data/1318605/000177134023000008/xslF345X05/edgardoc.xml

1/1

In [2]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [ ]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [ ]:

# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings.embed_documents, index, InMemoryDocstore({}), {})

In [ ]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI

In [ ]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=llm,
    memory=vectorstore.as_retriever(),
)
# Set verbose to be true
agent.chain.verbose = True